In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
 train = pd.read_csv('/content/drive/MyDrive/Expresso/Train1.csv')
test = pd.read_csv('/content/drive/MyDrive/Expresso/Test.csv')


In [ ]:
train.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [ ]:
train.isnull().sum()

user_id                 0
REGION             849299
TENURE                  0
MONTANT            756739
FREQUENCE_RECH     756739
REVENUE            726048
ARPU_SEGMENT       726048
FREQUENCE          726048
DATA_VOLUME       1060433
ON_NET             786675
ORANGE             895248
TIGO              1290016
ZONE1             1984327
ZONE2             2017224
MRG                     0
REGULARITY              0
TOP_PACK           902594
FREQ_TOP_PACK      902594
CHURN                   0
dtype: int64

In [ ]:
train.describe()


,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN
count,1.397309e+06,1.397309e+06,1.428000e+06,1.428000e+06,1.428000e+06,1.093615e+06,1.367373e+06,1.258800e+06,864032.000000,169721.000000,136824.000000,2.154048e+06,1.251454e+06,2.154048e+06
mean,5.532117e+03,1.152912e+01,5.510810e+03,1.836943e+03,1.397814e+01,3.366450e+03,2.776891e+02,9.541871e+01,23.109253,8.170132,7.553309,2.804251e+01,9.272461e+00,1.875474e-01
std,7.111339e+03,1.327407e+01,7.187113e+03,2.395700e+03,1.469403e+01,1.330446e+04,8.726889e+02,2.049873e+02,63.578086,41.169511,33.487234,2.228686e+01,1.228044e+01,3.903504e-01
min,1.000000e+01,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+03,2.000000e+00,1.000000e+03,3.330000e+02,3.000000e+00,0.000000e+00,5.000000e+00,7.000000e+00,2.000000,0.000000,0.000000,6.000000e+00,2.000000e+00,0.000000e+00
50%,3.000000e+03,7.000000e+00,3.000000e+03,1.000000e+03,9.000000e+00,2.570000e+02,2.700000e+01,2.900000e+01,6.000000,1.000000,2.000000,2.400000e+01,5.000000e+00,0.000000e+00
75%,7.350000e+03,1.600000e+01,7.368000e+03,2.456000e+03,2.000000e+01,2.895000e+03,1.560000e+02,9.900000e+01,20.000000,3.000000,5.000000,5.100000e+01,1.200000e+01,0.000000e+00
max,4.700000e+05,1.330000e+02,5.321770e+05,1.773920e+05,9.100000e+01,1.823866e+06,5.080900e+04,2.132300e+04,4174.000000,4792.000000,3697.000000,6.200000e+01,7.130000e+02,1.000000e+00


In [ ]:
X = train.drop(['CHURN', 'MRG', 'user_id', 'ZONE2', 'ZONE1'], axis = 1)
y = train['CHURN']

In [ ]:
len(X.columns)

14

In [ ]:
for column in X.columns:
  if X[column].dtype == 'O':
    X[column].fillna('missing', inplace = True)
  else:
    X[column].fillna(-99, inplace = True)

In [ ]:
top_pack = X['TOP_PACK'].unique()
top_packdf = pd.DataFrame(top_pack, columns= ['TOP_PACK'])
labelencoder = LabelEncoder()
top_packdf['TOP_PACK_CAT'] = labelencoder.fit_transform(top_packdf['TOP_PACK'])
top_packdf
X.loc[:, 'TOP_PACK_CAT'] = top_packdf['TOP_PACK_CAT']

In [ ]:
dummy = pd.get_dummies(X['REGION'])
dummy.head()
X = pd.concat([X, dummy], axis = 1)
X.head()

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,TOP_PACK_CAT,DAKAR,DIOURBEL,FATICK,KAFFRINE,KAOLACK,KEDOUGOU,KOLDA,LOUGA,MATAM,SAINT-LOUIS,SEDHIOU,TAMBACOUNDA,THIES,ZIGUINCHOR,missing
0,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,54,On net 200F=Unlimited _call24H,8.0,107.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,missing,I 18-21 month,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,4,missing,-99.0,135.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,missing,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,-99.0,90.0,46.0,7.0,17,On-net 1000F=10MilF;10d,1.0,108.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,62,"Data:1000F=5GB,7d",11.0,35.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,-99.0,39.0,24.0,-99.0,11,Mixt 250F=Unlimited_call24H,2.0,100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.skew()

MONTANT            4.616769
FREQUENCE_RECH    -0.480593
REVENUE            4.571837
ARPU_SEGMENT       4.509348
FREQUENCE         -0.518035
DATA_VOLUME       48.951166
ON_NET             9.793802
ORANGE             7.454566
TIGO               4.721151
REGULARITY         0.247408
FREQ_TOP_PACK     -0.211204
TOP_PACK_CAT       0.000000
DAKAR              1.228630
DIOURBEL           5.406000
FATICK             7.579636
KAFFRINE           6.783637
KAOLACK            4.388287
KEDOUGOU          45.921838
KOLDA              7.253742
LOUGA              4.335279
MATAM              8.430855
SAINT-LOUIS        3.876393
SEDHIOU           26.222581
TAMBACOUNDA        6.011504
THIES              3.009103
ZIGUINCHOR         9.755362
missing            0.432660
dtype: float64

In [ ]:
X.drop(['REGION', 'TOP_PACK'], axis = 1, inplace = True)

In [ ]:
X.nunique()

TENURE                8
MONTANT            6541
FREQUENCE_RECH      124
REVENUE           38115
ARPU_SEGMENT      16536
FREQUENCE            92
DATA_VOLUME       41551
ON_NET             9885
ORANGE             3168
TIGO               1316
REGULARITY           62
FREQ_TOP_PACK       246
TOP_PACK_CAT        141
DAKAR                 2
DIOURBEL              2
FATICK                2
KAFFRINE              2
KAOLACK               2
KEDOUGOU              2
KOLDA                 2
LOUGA                 2
MATAM                 2
SAINT-LOUIS           2
SEDHIOU               2
TAMBACOUNDA           2
THIES                 2
ZIGUINCHOR            2
missing               2
dtype: int64

In [ ]:
cat_col = ['TENURE']
for column in cat_col:
  X[column], _ = X[column].factorize()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1994, stratify = y)

In [ ]:
X.head()

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,FREQ_TOP_PACK,TOP_PACK_CAT,DAKAR,DIOURBEL,FATICK,KAFFRINE,KAOLACK,KEDOUGOU,KOLDA,LOUGA,MATAM,SAINT-LOUIS,SEDHIOU,TAMBACOUNDA,THIES,ZIGUINCHOR,missing
0,0,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,54,8.0,107.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,4,-99.0,135.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,3600.0,2.0,1020.0,340.0,2.0,-99.0,90.0,46.0,7.0,17,1.0,108.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,62,11.0,35.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1000.0,1.0,985.0,328.0,1.0,-99.0,39.0,24.0,-99.0,11,2.0,100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.dtypes

TENURE              int64
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
TIGO              float64
REGULARITY          int64
FREQ_TOP_PACK     float64
TOP_PACK_CAT      float64
DAKAR               uint8
DIOURBEL            uint8
FATICK              uint8
KAFFRINE            uint8
KAOLACK             uint8
KEDOUGOU            uint8
KOLDA               uint8
LOUGA               uint8
MATAM               uint8
SAINT-LOUIS         uint8
SEDHIOU             uint8
TAMBACOUNDA         uint8
THIES               uint8
ZIGUINCHOR          uint8
missing             uint8
dtype: object

In [ ]:
model=CatBoostClassifier(iterations=1000, learning_rate=0.08, depth=6, random_seed=2021, eval_metric='AUC')
model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100,)

0:	test: 0.9257178	test1: 0.9260853	best: 0.9260853 (0)	total: 607ms	remaining: 10m 6s
100:	test: 0.9310593	test1: 0.9312151	best: 0.9312151 (100)	total: 1m 5s	remaining: 9m 40s
200:	test: 0.9314437	test1: 0.9314995	best: 0.9314995 (200)	total: 2m 8s	remaining: 8m 30s
300:	test: 0.9317032	test1: 0.9316217	best: 0.9316217 (300)	total: 3m 11s	remaining: 7m 25s
400:	test: 0.9318960	test1: 0.9316864	best: 0.9316868 (397)	total: 4m 14s	remaining: 6m 20s
500:	test: 0.9320623	test1: 0.9317252	best: 0.9317278 (494)	total: 5m 17s	remaining: 5m 16s
600:	test: 0.9322135	test1: 0.9317436	best: 0.9317464 (590)	total: 6m 20s	remaining: 4m 12s
700:	test: 0.9323490	test1: 0.9317596	best: 0.9317609 (695)	total: 7m 24s	remaining: 3m 9s
800:	test: 0.9324691	test1: 0.9317595	best: 0.9317633 (768)	total: 8m 26s	remaining: 2m 5s
999:	test: 0.9327001	test1: 0.9317698	best: 0.9317700 (953)	total: 10m 32s	remaining: 0us

bestTest = 0.9317699945
bestIteration = 953

Shrink model to first 954 iterations.


In [ ]:
test.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,THIES,K > 24 month,5000.0,5.0,5000.0,1667.0,5.0,NaN,378.0,11.0,5.0,NaN,NaN,NO,42,On-net 1000F=10MilF;10d,5.0
1,000055d41c8a62052dd426592e8a4a3342bf565d,NaN,I 18-21 month,300.0,2.0,326.0,109.0,3.0,397.0,NaN,0.0,NaN,NaN,NaN,NO,41,"Data: 100 F=40MB,24H",1.0
2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,DAKAR,K > 24 month,3300.0,25.0,3400.0,1133.0,26.0,7150.0,0.0,2.0,5.0,NaN,NaN,NO,57,"Data: 100 F=40MB,24H",22.0
3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,9,NaN,NaN
4,0000bae5480628cf8fe51ad84bcb39772fc79224,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,10,NaN,NaN


In [ ]:
X_test = test.drop(['user_id', 'MRG', 'ZONE1', 'ZONE2'], axis = 1)
ID = test['user_id']

In [ ]:
for column in X_test.columns:
  if X_test[column].dtype == 'O':
    X_test[column].fillna('missing', inplace = True)
  else:
    X_test[column].fillna(-99, inplace = True)

In [ ]:
top_pack = X_test['TOP_PACK'].unique()
top_packdf = pd.DataFrame(top_pack, columns= ['TOP_PACK'])
labelencoder = LabelEncoder()
top_packdf['TOP_PACK_CAT'] = labelencoder.fit_transform(top_packdf['TOP_PACK'])
top_packdf
X_test.loc[:, 'TOP_PACK_CAT'] = top_packdf['TOP_PACK_CAT']


In [ ]:
dummy = pd.get_dummies(X_test['REGION'])
dummy.head()
X_test = pd.concat([X_test, dummy], axis = 1)

In [ ]:
cat_col = ['TENURE']
for column in cat_col:
  X_test[column], _ = X_test[column].factorize()

In [ ]:
X_test.drop(labels = ['REGION', 'TOP_PACK'], axis = 1, inplace = True)

In [ ]:
model.fit(X, y)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	total: 464ms	remaining: 7m 43s
1:	total: 928ms	remaining: 7m 43s
2:	total: 1.42s	remaining: 7m 51s
3:	total: 1.89s	remaining: 7m 51s
4:	total: 2.4s	remaining: 7m 57s
5:	total: 2.89s	remaining: 7m 59s
6:	total: 3.38s	remaining: 8m
7:	total: 3.87s	remaining: 7m 59s
8:	total: 4.38s	remaining: 8m 2s
9:	total: 4.85s	remaining: 8m
10:	total: 5.33s	remaining: 7m 59s
11:	total: 5.81s	remaining: 7m 58s
12:	total: 6.29s	remaining: 7m 57s
13:	total: 6.75s	remaining: 7m 55s
14:	total: 7.23s	remaining: 7m 54s
15:	total: 7.69s	remaining: 7m 52s
16:	total: 8.16s	remaining: 7m 51s
17:	total: 8.62s	remaining: 7m 50s
18:	total: 9.12s	remaining: 7m 50s
19:	total: 9.58s	remaining: 7m 49s
20:	total: 10s	remaining: 7m 47s
21:	total: 10.5s	remaining: 7m 46s
22:	total: 10.9s	remaining: 7m 44s
23:	total: 11.4s	remaining: 7m 43s
24:	total: 11.9s	remaining: 7m 43s
25:	total: 12.3s	remaining: 7m 42s
26:	total: 12.8s	remaining: 7m 40s
27:	total: 13.3s	remaining: 7m 41s
28:	total: 13.7s	remaining: 7m 39s
29:	tot

In [ ]:
my_sub = pd.DataFrame(data = {'user_id' : ID, 'CHURN' : model.predict_proba(X_test)[:, 1]})

In [ ]:
my_sub.to_csv('second submission', index = False)

In [ ]:
my_sub.head()

,user_id,CHURN
0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,0.001163
1,000055d41c8a62052dd426592e8a4a3342bf565d,0.167287
2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,0.000814
3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,0.439407
4,0000bae5480628cf8fe51ad84bcb39772fc79224,0.413996
